In [ ]:
#Coded with <3 by BerkayDemirel
import gym
import numpy as np
import numpy.random as random
import pandas as pd
import cv2
import math  
from minisom import MiniSom
from matplotlib import pyplot as plt
from matplotlib import cm as cm
from PIL import Image

In [ ]:
batch = [] #Batch of observations
maxhistory = []
obshistory = [] #Observation history
acthistory = [] #Action history
goldhistory=[] #Reward history
fskip = 3 #Frameskip

envs =["SpaceInvadersDeterministic-v4", "SpaceInvadersNoFrameskip-v4","SpaceInvaders-v4",
       "MontezumaRevengeNoFrameskip-v4", "SeaquestNoFrameskip-v4", "MsPacmanNoFrameskip-v4"]

# Deterministic skips 3 frames for Space Invaders (4 for the rest of the atari games)
# NoFrameskip does not skip frames (Requires frameskip/maxing to see all elements on the screen)
# Normal skips between 2 to 5 frames randomly (Unreliable)

env = gym.make(envs[1])
actions = env.unwrapped.get_action_meanings()
for i_episode in range(4):
    observation = env.reset()
    for t in range(1000):
        env.render()
        action = random.randint(2,4)#env.action_space.sample()
        for i in range(fskip): #Repeat same action fskip times and record everything
            observation, reward, done, info = env.step(action)
            #analyseframe(observation)
            batch.append(observation)
            obshistory.append(observation)
            acthistory.append(actions[action])
            goldhistory.append(reward)
        f = getmaxed(batch)
        maxhistory.append(f)
        batch = []
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

In [ ]:
print(len(obshistory), obshistory[0].shape)
env.unwrapped.get_action_meanings()

In [ ]:
for i in range(1005,1020,fskip):
    plt.imshow(np.array(np.squeeze(obshistory[i])))
    plt.title("Frame " + str(i) + " Act: " + acthistory[i])
    plt.show()

### Preprocessing Images 

In [ ]:
preprocessed=[]
preprocessed_=[]
preprocessedx=[]

#Preprocessing by reducing channels, cropping, resizing and normalizing the images
def preprocess(observation):
    observation = cv2.cvtColor(cv2.resize(observation, (84, 110)), cv2.COLOR_BGR2GRAY)
    observation = observation[26:110,:]
    ret, observation = cv2.threshold(observation,1,255,cv2.THRESH_BINARY)
    return np.reshape(observation,(84,84,1))

#Preprocessing without channel reduction and normalization
def preprocess_(observation):
    observation = cv2.resize(observation, (84, 110))
    observation = observation[26:110,:]
    return np.reshape(observation, (84,84,3))

#Preprocessing only by cropping
def preprocessx(observation):
    observation = observation[25:195,:]
    return observation

x = 18
exp = preprocess(obshistory[x])
exp_= preprocess_(obshistory[x])
expx = preprocessx(obshistory[x])

print("Before processing: " + str(np.array(obshistory[x]).shape))
plt.imshow(np.array(np.squeeze(obshistory[x])))
plt.title("Frame " + str(x))
plt.show()

print("After processing: " + str(np.array(exp).shape))
plt.imshow(np.array(np.squeeze(exp)))
plt.title("Frame " + str(x))
plt.show()

print("After processing_: " + str(np.array(exp_).shape))
plt.imshow(np.array(np.squeeze(exp_)))
plt.title("Frame " + str(x))
plt.show()
                               
                               
print("After processingx: " + str(np.array(expx).shape))
plt.imshow(expx)
plt.title("Frame " + str(x))
plt.show()

for i in range(len(obshistory)):
    preprocessed.append(preprocess(obshistory[i]))
    preprocessed_.append(preprocess_(obshistory[i]))
    preprocessedx.append(preprocessx(obshistory[i]))

### Maxpool
As the atari hardware was limited to showing only a certain number of objects on the screen, this allows us to capture all the elements present in the screen for a human viewer by taking the max of (n) consequent frames.

In [ ]:
# Get the maximum values from (fskip) matrices in order to account for the skipped objects in the frames :
batch=[]

#Inputs an array, outputs single observation
def getmaxed(obs,skip): 
    q = obs[0]
    for i in range(len(obs)):
        q = np.fmax(q, obs[i])
    return q

#Inputs an array, outputs an array, used for batch processing
def getmax(obs,frame):  

    for i in range(len(obs)//frame):
        cache=[]
        q=None
        for k in range(frame):
            cache.append(obs[i*frame+k])
        q = cache[0]
        for e in range(len(cache)-1):
            q = np.fmax(q, cache[e+1])
        batch.append(q)

    return batch

maxed = getmax(preprocessedx,3)

len(maxed), maxed[1].shape

In [ ]:
#Plot frames before and after maxpooling

for i in range(0,10):
    plt.figure(figsize=(40,10))
    
    plt.subplot(151)
    plt.imshow(np.array(np.squeeze(preprocessedx[i*fskip])))
    plt.title("Before Maxpool Frame " + str(i*fskip))
    
    plt.subplot(152)
    plt.imshow(np.array(np.squeeze(preprocessedx[i*fskip+1])))
    plt.title("Before Maxpool Frame " + str(i*fskip+1))
    
    plt.subplot(153)
    plt.imshow(np.array(np.squeeze(preprocessedx[i*fskip+2])))
    plt.title("Before Maxpool Frame " + str(i*fskip+2))

    plt.subplot(154)
    plt.imshow(np.array(np.squeeze(preprocessedx[i*fskip+3])))
    plt.title("Before Maxpool Frame " + str(i*fskip+3))
    
    plt.subplot(155)
    plt.imshow(np.array(np.squeeze(preprocessedx[i*fskip+4])))
    plt.title("Before Maxpool Frame " + str(i*fskip+4))
    plt.show()
    plt.imshow(np.array(np.squeeze(maxed[i])))
    plt.title("After Maxpool")
    plt.show()

### Blob Detection

In [ ]:
def GetKeys(exp):
    
    keypoints=[]
    
    # Setup SimpleBlobDetector parameters.
    params = cv2.SimpleBlobDetector_Params()

    # Change thresholds
    params.minThreshold = 0
    params.maxThreshold = 20 # 0-20 is ideal for SpaceInvaders

    # Filter by Area
    params.filterByArea = False
    params.minArea = 4

    # Filter by Circularity
    params.filterByCircularity = False
    params.minCircularity = 0.1

    # Filter by Convexity
    params.filterByConvexity = False
    params.minConvexity = 0.87

    # Filter by Inertia
    params.filterByInertia = True
    params.minInertiaRatio = 0.01    #True, with a value of 0.01 is ideal for SpaceInvaders

    # Filter by Colour
    params.filterByColor = True   
    params.blobColor = 255   #True is ideal for SpaceInvaders, may not work properly in envs with rich backgrounds!

    # Check opencv version and create a detector with the set parameters
    is_v2 = cv2.__version__.startswith("2.")
    if is_v2:
        detector = cv2.SimpleBlobDetector()
    else:
        detector = cv2.SimpleBlobDetector_create(params)

    # Detect blobs.
    keypoints = detector.detect(exp)

    # Draw detected blobs as red circles.
    # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures
    # the size of the circle corresponds to the size of blob

    im_with_keypoints = cv2.drawKeypoints(exp, keypoints, np.array([]), (250,0,0), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    #im_with_keypoints = cv2.circle(ex3, keypoints, 0, (200,200,200),1)
    #Show keypoints
    #print(keypoints)
    
    # cv2.imshow("Keypoints", im_with_keypoints)
    cv2.imwrite("Keypoints.png", im_with_keypoints)
    #For Debugging: To show the image with found keypoints:
    #plt.figure(figsize=(5,10))
    #plt.imshow(im_with_keypoints)
    return keypoints

### Canny Edge Detection

In [ ]:
imgedge= obshistory[480]
imgedge2= np.array(np.squeeze(maxed[830]))

edges = cv2.Canny(imgedge,10,10)
edges2 = cv2.Canny(imgedge2, 10,10)

plt.figure(figsize=(10,5))
plt.subplot(121),plt.imshow(imgedge,cmap = 'hot')
plt.title('Original Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(edges,cmap = 'gray')
plt.title('Edge Image'), plt.xticks([]), plt.yticks([])
plt.show()

plt.figure(figsize=(10,5))
plt.subplot(121), plt.imshow(imgedge2,cmap="hot")
plt.title("B&W Original Image"), plt.xticks([]), plt.yticks([])
plt.subplot(122), plt.imshow(edges2, cmap="gray")
plt.title("Edge Image"), plt.xticks([]), plt.yticks([])
plt.show()

### Flow Detection Using Features

In [ ]:
#Flow detection parameters
feature_params = dict( maxCorners = 100,qualityLevel = 0.3, minDistance = 2, blockSize = 5 )
lk_params = dict( winSize = (10,10),maxLevel = 2,criteria = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 10, 0.03))

#Some other corner/feature finders:

#p1 = cv2.goodFeaturesToTrack(maxed[452], mask = None, **feature_params) #Finds corners but is not reliable
#p2 = cv2.Canny(maxed[0], 10 , 1) #Finds edges but is not reliable
#p3 = cv2.cornerEigenValsAndVecs(maxed[0], 3, 3)  #Another corner finder


#Get the flow from img1 to img2, using the points/features p0, any of the above can be fed as p0
def getflow(img1, img2, p0): 
    
    #Calculate flow
    p1, st, err = cv2.calcOpticalFlowPyrLK(img1, img2, p0, None, **lk_params)

    #Create a mask to show the flow
    mask = np.zeros_like(img1)

    #Find the flow movements
    good_new = p1[st==1]
    good_old = p0[st==1]
    notfound_new = p1[st==0]
    #Uncomment for colour use in flow display
    #color = np.random.randint(0,255,(2,3))
    #cv2.line(img, pt1, pt2, color[, thickness[, lineType[, shift]]])
    
    #For debugging: print(preprocessed[1].shape, mask.shape)

    #Create flow display
    for i,(new,old) in enumerate(zip(good_new,good_old)):
        a,b = new.flatten() # .ravel() is faster but changes the original (Consider for production?)
        c,d = old.flatten()
        mask = cv2.line(mask, (a,b),(c,d), (100,100,100), 1)
        frame = cv2.circle(img2,(c,d),0,(200,200,200),1)
        img = cv2.add(img2,mask)

    #Save flow images
    img = cv2.add(frame,mask)
    cv2.imwrite('mixed.png',img)
    cv2.imwrite('flow.png',mask)
    cv2.imwrite('origin.png',frame)
    cv2.imwrite("new.png", img2)
    cv2.imwrite("old.png", img1)
    
    return good_old, good_new, st

In [ ]:
#For Using Blob Keypoints as Features for Flow Calculation:

#Transform keypoints to features:
def GetFeatures(keypoints):
    
    corx=[]
    cory=[]
    for keypoint in keypoints:
        corx.append(keypoint.pt[0])
        cory.append(keypoint.pt[1])

    corrx = np.array(corx)
    corry = np.array(cory)
    corrnew = np.column_stack((corrx,corry))
    corrnew = corrnew[:, np.newaxis].astype("float32")
    print(np.shape(corrnew))
    return corx, cory, corrnew

#For cropping the features:

def centeredCrop(img, size, xval, yval):

    xval = np.ceil(xval)
    yval = np.ceil(yval)
    
    left = np.ceil(xval-size/2)
    right = np.ceil(xval+size/2)

    top = np.ceil(yval-size/2)
    bottom = np.ceil(yval+size/2)
    
    #For Debugging: print(left-right),print(top-bottom)

    cImg = img[int(top):int(bottom), int(left):int(right)]
    height, width, channels = cImg.shape
    #For Debugging : print(height,width)
    if height < size:
        cImg = cv2.copyMakeBorder(cImg, 0, (size-height), 0, 0, cv2.BORDER_CONSTANT,value=[0, 0, 0])
    if width < size:
        cImg = cv2.copyMakeBorder(cImg,0, 0, 0, (size-width), cv2.BORDER_CONSTANT, value=[0,0,0])
    return cImg


### Finding Self through Flow - Action Coherence

In [ ]:
k = 1400

columns = ("Entity", "Movement", "Direction", "Action", "Congruence with Action")
df = pd.DataFrame(columns=columns)
#df = df.fillna(0) # Fill empty with 0s rather than NaNs

keypoints = GetKeys(preprocessedx[k])
corx, cory, corrnew = GetFeatures(keypoints)

z = getflow(preprocessedx[k],preprocessedx[k+5], corrnew)
#Getting the flow for more than 100 frames may cause skipping of movements/ merging movements together & other glitches

print(acthistory[k])

actbinary = []
movebinary = []
congruence = []

for i in range(len(acthistory)):
    if acthistory[i] == "RIGHT":
        actbinary.append(1)
    elif acthistory[i] == "LEFT":
        actbinary.append(0)
    else:
        print("Unknown action!")
        
#print(z[1]-z[0])

for i in range(len(z[1])):
    move = z[1][i] - z[0][i]
    
    if move[0] > 0.1:
        print(str(i)+ "th data point is moving right by " + str(move[0]))
        movebinary.append(1)
        if actbinary[k] ==1:
            congruence.append(1)
        else:
            congruence.append(0)
        
        plt.title("Right")
        plt.imshow(centeredCrop(preprocessedx[k],12,corx[i], cory[i]))
        plt.show()
    elif move[0]<-0.1:
        print(str(i)+ "th data point is moving left by " + str(-move[0]))
        movebinary.append(-1)
        if actbinary[k]==-1:
            congruence.append(1)
        else:
            congruence.append(0)
        
        plt.title("Left")
        plt.imshow(centeredCrop(preprocessedx[k],12,corx[i], cory[i]))
        plt.show()

    else:
        print(str(i)+ "th data point is not moving.")
        movebinary.append(0)
        congruence.append(0)
        plt.title("Standing")
        plt.imshow(centeredCrop(preprocessedx[k],12,corx[i], cory[i]))
        plt.show()
    move = []
    

In [ ]:
len(acthistory), len(preprocessedx), len(actbinary), len(congruence)

In [ ]:
for i in range(len(z[1])):
    df = df.appenddf({
    "Entity": i,
    "Movement": 0 ,
    "Direction": actbinary[k],
    "Action": , 
    "Congruence": ,
    }, ignore_index=True)

### Preparing frames for Som Training

In [ ]:
z = 3300
selfdata = []
data = []
datacolor = []
dataset = []
dataset_color = []

def CropFeatures(frame):
    dataset=[]
    dataset_mono=[]
    dataset_color=[]
    corx,cory = [],[0]
    keypoints = GetKeys(frame)
    corx, cory, corrnew = GetFeatures(keypoints)
    for i in range(len(corx)):
        aaa = centeredCrop(frame, 12 , corx[i], cory[i])
        dataset_color.append(aaa)
        aaa = cv2.cvtColor(aaa, cv2.COLOR_BGR2GRAY)
        dataset_mono.append(aaa)
        dataset.append(aaa.flatten().tolist())
        #plt.imshow(aaa)
        #plt.show()
    return dataset, dataset_color, corx, cory

for i in range(0,100,20):
    data, datacolor, corx, cory = CropFeatures(preprocessedx[i])
    print(len(data)), print(len(datacolor))
    dataset += data
    dataset_color += datacolor
#CropFeatures(preprocessedx[z])
#print(len(dataset)),print(len(dataset_color))

### Som Training

In [ ]:
#dataset = CropFeatures(preprocessedx[z])
def Trainsom (dataset, iters):
    global som
    som = MiniSom(4, 4, 144, sigma=0.3, learning_rate=0.1) # Initialization of NxN SOM
    #print ("Training...")
    som.train_random(dataset, 10000) # Trains the SOM with (n) iterations
    print ("Done!")
    print ("\nData is clustered as: ")
    print(som.activation_response(dataset))
    print("\nData Category Visualization: ")
    for i in range(len(dataset)):   
        winner = som.winner(dataset[i])
        plt.title("Entity "+ str(winner))
        plt.imshow(dataset_color[i])
        plt.show()
    
Trainsom(dataset,10000)    
#For Debugging: som.winner(dataset[x]), som.quantization_error(dataset)

In [ ]:
#Search through the whole image for entities

sees = []
lookat = []
step = 6

self = (2,2)
Enemy A = (0,2)
Enemy B = (3,3)
Enemy C = (0,0)
Enemy D = (2,0)
Enemy E = (1,3)
Block = (3,1)

for i in range(step,160,step//2):
    for k in range(step,170,step//2):
        val = (i,k)
        lookat.append(val)

for i in range(len(lookat)):
    aaa = centeredCrop(preprocessedx[940],12,lookat[i][0], lookat[i][1])
    sees.append(aaa)
    aaa = cv2.cvtColor(aaa, cv2.COLOR_BGR2GRAY)
    aaa = aaa.flatten().tolist()
    identity = som.winner(aaa)
    plt.title("Entity "+ str(identity)+ " @ " + str(lookat[i]))
    plt.imshow(sees[i])
    plt.show()


In [ ]:
keypoints = GetKeys(preprocessedx[1080])
corx, cory, corrnew = GetFeatures(keypoints)

dataset=[]
dataset_mono=[]
dataset_color=[]

for i in range(len(corx)):
    aaa = centeredCrop(preprocessedx[1080],12,corx[i], cory[i])
    aaa2 = cv2.cvtColor(aaa, cv2.COLOR_BGR2GRAY)
    baa = aaa2.flatten().tolist()
    plt.title(str(som.winner(baa)))
    plt.imshow(aaa)
    plt.show()

In [ ]:
#Alternative Som training with progress messages
max_iter = 10000
q_error_pca_init = []
iter_x = []
for i in range(max_iter):
    percent = 100*(i+1)/max_iter
    rand_i = np.random.randint(len(dataset))
    som.update(dataset[rand_i], som.winner(dataset[rand_i]), i, max_iter)
    if (i+1) % 1000 == 0:
        error = som.quantization_error(dataset)
        q_error_pca_init.append(error)
        iter_x.append(i)
        print(f'\riteration={i:2d} status={percent:0.2f}% error={error}')

In [ ]:
fr=0
def analyseframe(frame):
    global fr
    global df
    pre = preprocessx(frame)
    newdata, newdata_color, corx, cory = CropFeatures(pre)
    print(len(newdata), len(newdata_color), len(corx), len(cory))
    fr+=1
    for i in range(len(newdata)):   
        winner = som.winner(newdata[i])
        df = df.append({'Frame':fr, 'Id':winner, "Pic": newdata_color[i], "Xpos":corx[i], "Ypos":cory[i],"Movement":0, "Self":0}, ignore_index=True)
        #plt.title("Entity "+ str(winner) + " @" + str(corx[i]) +" , "+str(cory[i]) )
        #plt.imshow(newdata_color[i])
        #plt.show()
    print("Frame " + str(fr) + " analysed.")


In [ ]:
columns = ["Frame","Id", "Pic", "Xpos", "Ypos", "Movement", "Self"]
df = pd.DataFrame(columns=columns)
#df = df.fillna(0) # with 0s rather than NaNs

In [ ]:
df.loc[df['Id'] == (2,2)]

In [ ]:
plt.figure(figsize=(30,20))
plt.xlim(0,160)
plt.ylim(170,0)
colors = np.where(df["Id"]==(1,1),'r','b')
plt.scatter(df.Xpos, df.Ypos, c=colors, s=40, marker="*")